<p style="text-align: center;font-size: 40pt">Uncertainty in 3D</p>

In [ ]:
%matplotlib widget
#%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
from matplotlib import cm

import numpy as np

%run ./scripts/helper_func.py
path = "{0}/lessons/1-transformations_2d/scripts/helper_func.py".format(get_root_path())
%run $path
path = "{0}/lessons/3-transformations_3d/scripts/helper_func.py".format(get_root_path())
%run $path
path = "{0}/common/scripts/style.py".format(get_root_path())
%run $path

# Overview 

Requirement
- [Descriptive statistics](5.1-lesson_descriptive_statistics.ipynb)
- [Rotation in 3D - matrix](../3-transformations_3d/3.2-lesson_rotation_mat.ipynb)

Objectives of this lesson:

- introduce multivariate normal distributions
- highlight drawing misconceptions 
- explain covariance matrices with a geometric meaning


Hidden custom latex commands here $ \curvearrowright$

----
[comment]: <> (General commands)
$\newcommand{\textcomma}{\quad\text{,}}$
$\newcommand{\textdot}{\quad\text{.}}$
$\newcommand{\vec}[1]{\overrightarrow{#1}}$
$\newcommand{\mat}[1]{\mathbf{#1}}$
$\newcommand{\frame}[1]{\mathcal{#1}}$
$\newcommand{\point}[2][]{{}^{#1}\mathbf{#2}}$
$\newcommand{\pointsym}[2][]{{}^{#1}\boldsymbol{#2}}$
$\newcommand{\matsym}[1]{\boldsymbol{#1}}$
$\newcommand{\real}{\mathbb{R}}$
$\newcommand{\bmat}[1]{\begin{bmatrix}#1\end{bmatrix}}$
$\newcommand{\F}[2][]{{}_{#2}^{#1}\mathscr{F}}$
$\newcommand{\Fmat}[2][]{{}_{#2}^{#1}\mat{F}}$
$\newcommand{\origin}[2][]{{}_{#2}^{#1}\mat{o}}$
$\newcommand{\T}[2][]{{}_{#2}^{#1}\mat{T}}$
$\newcommand{\t}[2][]{{}_{#2}^{#1}\mat{t}}$
$\newcommand{\R}[2][]{{}_{#2}^{#1}\mat{R}}$
$\newcommand{\f}{\vec{\mathscr{f}}}$
$\newcommand{\ax}[2][]{{}_{#2}^{#1}\vec{\mathscr{x}}}$
$\newcommand{\ay}[2][]{{}_{#2}^{#1}\vec{\mathscr{y}}}$
$\newcommand{\az}[2][]{{}_{#2}^{#1}\vec{\mathscr{z}}}$
$\newcommand{\aw}[2][]{{}_{#2}^{#1}\vec{\mathscr{w}}}$
$\newcommand{\axi}{\mathscr{x}}$
$\newcommand{\ayi}{\mathscr{y}}$
$\newcommand{\azi}{\mathscr{z}}$
$\newcommand{\awi}{\mathscr{w}}$
$\newcommand{\pointx}[2][]{{}^{#1}{#2}_{\axi}}$
$\newcommand{\pointy}[2][]{{}^{#1}{#2}_{\ayi}}$
$\newcommand{\pointz}[2][]{{}^{#1}{#2}_{\azi}}$
$\newcommand{\SO}[1]{\mathrm{SO}(#1)}$
$\newcommand{\var}{\text{Var}}$
----

# Gaussian vs normal 

A Gaussian function $f(x)$ uses the exponential function with an amplitude $a$, a location $b$, and a spread $c$ in the following way:

\begin{aligned}
f(x) = a \exp\left( -\frac{1}{2} \frac{(x-b)^2}{c^2} \right)
\textcomma
\end{aligned}

with the constraint that $c > 0$ because of the division.
The normal distribution density function $\phi(x)$ is build over the Gaussian function, with the additional constraint that its area under the curve (i.e., its integral) must equal one.
This is a common constraint to all probability distributions. 
If we have a set $\mathcal{X} = \{x_1, x_2, \dots, x_i\}$ for which we state that $\mathcal{X} \sim \mathcal{N}(\mu, \sigma^2)$, we are saying that the values $x_i$ in $\mathcal{X}$ are coming from a normal distribution with a mean $\mu$ and a variance $\sigma^2$.
More precisely, the density function of a normal distribution is defined as follow

\begin{aligned}
\phi(x) = \frac{1}{\sigma\sqrt{2\pi}} \exp \left( -\frac{1}{2}\frac{(x-\mu)^2}{\sigma^2} \right)
\textdot
\end{aligned}

The following graph shows that Gaussian function are not limited in amplitude, which is not the case for the density function.


In [ ]:
def gaussian(x, a, b, c):
    return a * np.exp(-0.5 * ((x - b)**2)/(c**2))

def normal(x, mu, sigma):
    a = 1/(sigma*np.sqrt(2.*np.pi))
    return a * np.exp(-0.5 * ((x - mu)**2)/(sigma**2))


x = np.arange(-4, 4, 0.01)
f = gaussian(x, a=1., b=0., c=1.)
n = normal(x, mu=0., sigma=1.)

#--------------------------------
# plotting
if 'fig' in globals():
    plt.close(fig)
fig = plt.figure(figsize=(8,4))
ax = fig.add_subplot(111)

name = "Gaussian functions"
for a in np.arange(2,0, -0.1):
    ax.plot(x, gaussian(x, a, b=0., c=1.), alpha = a/2., color="tab:red", label=name)
    if name:
        name = ""

ax.plot(x, n, '--', label="Normal distribution")
ax.set_ylim(0,1)
ax.legend()
pretty_ax(ax, 'Comparison', 'Density', 'Values of $x_i$')
fig.tight_layout()

# Multivariate normal distribution

We will do few modifications to the well know normal distribution equation to ease the transition to multiple dimensions.
To avoid notation problems, we will change the name of our set to $\mathcal{A}$ instead of $\mathcal{X}$.
To avoid a squared value, we will first use another term for the variance, so we have
\begin{aligned}
\text{Var}(\mathcal{A}) = \sigma^2 
\textdot
\end{aligned}

Then, let's say that we don't like division and that you recall that $\frac{1}{c} = c^{-1}$, we can express the density function as

\begin{aligned}
\phi(a) 
&= \frac{1}{\sigma\sqrt{2\pi}} \exp \left( -\frac{1}{2}\frac{(a-\mu)^2}{\sigma^2} \right)\\
&= \frac{1}{\sqrt{2\pi \var(\mathcal{A}) }} 
\exp \left( -\frac{1}{2} \frac{(a - \mu)(a - \mu)}{\var(\mathcal{A})}  \right) \\
&= \frac{1}{ {\left( 2\pi \var(\mathcal{A}) \right)^\frac{1}{2}}} 
\exp \left( -\frac{1}{2} (a - \mu) \var(\mathcal{A})^{-1} (a - \mu)  \right) \\
&= \Big(2\pi \var(\mathcal{A}) \Big)^{-\frac{1}{2}} 
\exp \left( -\frac{1}{2} (a - \mu) \var(\mathcal{A})^{-1} (a - \mu)  \right) 
\textdot
\end{aligned}

There is no division for matrices, so although this last form of the equation looks ugly, it is very close to the multivariate version of a normal distribution, which is as follow

\begin{aligned}
\phi(\mat{a}) = 
\Big( (2\pi)^k \det \left( \matsym{\Sigma} \right)  \Big)^{-\frac{1}{2}}
\exp \left(-\frac{1}{2} (\mat{a}-\matsym{\mu})^\mathrm{T} \matsym{\Sigma}^{-1} (\mat{a}-\matsym{\mu}) \right)
\textcomma
\end{aligned}

where we have a coordinate $\mat{a} \in \real^{k}$, a mean vector $\matsym{\mu} \in \real^{k}$ and a covariance matrix $\matsym{\Sigma} \in \real^{k \times k}$, along with $k$ being the number of dimensions.
The covariance matrix $\matsym{\Sigma}$ has more constraints to it, but we will cover them properly in the next section.
For now, just focus on the big picture.
If we do an example in 3D, so that $k=3$ those quantities would look like this:

\begin{aligned}
\mat{a} = \bmat{a_\axi \\ a_\ayi \\ a_\azi}, \quad
\matsym{\mu} = \bmat{\mu_\axi \\ \mu_\ayi \\ \mu_\azi}, \quad
\matsym{\Sigma} = \bmat{
\cdot & \cdot & \cdot \\
\cdot & \cdot & \cdot \\
\cdot & \cdot & \cdot \\
}
\textdot
\end{aligned}

You should also note that the density function $\phi(\mat{a})$ still output a scalar.
Let see how this equation behaves for $k = \{1, 2, 3\}$.
The following graphs show different ways normal distributions can be displayed.
The code is large and uses many concept not yet explained, so just focus on the graph for now.
Each column represent a normal distribution in 1D, 2D, and 3D.
The first row uses an extra dimension to show the density.
The second row represents density on a dense field with color and marker size changing with density.
Finally, the third row show the typical simplification people are doing when displaying these distributions.

In [ ]:

res = 0.3
lim = 5.
sig_gain = 3. # how many sigma should we put the border

x = np.arange(-lim, lim, res)

# 1D - prepare data
X_1d = x
mu_1d = np.array([[0.]])
cov_1d = np.array([[1.]])
phi_1d = normal_density(X_1d, mu_1d, cov_1d)

# 2D - prepare data
xx_2d, yy_2d = np.meshgrid(x, x)
X_2d = np.vstack([xx_2d.flatten(), yy_2d.flatten()])

mu_2d = np.array([0.,0.])

theta = np.pi/4.
R = rot_2d(theta)
Lambda = scale_mat([0.6, 1.3])
cov_2d = R @ Lambda @ R.T
phi_2d = normal_density(X_2d, mu_2d, cov_2d)
zz_2d = np.reshape(phi_2d, xx_2d.shape)

# 2D - prepare data
x_sub = x[::3]
xx_3d, yy_3d, zz_3d = np.meshgrid(x_sub, x_sub, x_sub)
X_3d = np.vstack([xx_3d.flatten(), yy_3d.flatten(), zz_3d.flatten()])

theta = np.pi/3.
R = rotation_matrix_y(theta)
Lambda = scale_mat([0.6, 2.3, 2.2])

mu_3d = np.array([0., 0., 0.])
cov_3d = R @ Lambda @ R.T
phi_3d = normal_density(X_3d, mu_3d, cov_3d)

#--------------------------------
# plotting
if 'fig' in globals():
    plt.close(fig)
fig = plt.figure(figsize=(8,8))

# 1D - density
ax = fig.add_subplot(331)
ax.plot(x, phi_1d)
pretty_ax(ax, '1D', 'Density', 'Values of $x_i$')
if True:
    ax.spines['bottom'].set_position('zero')
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.get_xaxis().set_ticklabels([])
    ax.set_xlim(-lim,lim)
    pass

# 1D - field
ax = fig.add_subplot(334)
ax.scatter(x, np.zeros_like(x), c=phi_1d, s=phi_1d*100., zorder=3)
ax.set_ylabel("Field")
if True:
    ax.spines['bottom'].set_position('zero')
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.get_xaxis().set_ticklabels([])
    ax.set_xlim(-lim,lim)
    ax.set_ylim(-lim,lim)
    pass

# 1D - simplify
ax = fig.add_subplot(337)
dist = sig_gain * np.sqrt(cov_1d)/2.
ax.annotate('', xy=(mu_1d + dist, 0),
             xytext=(mu_1d - dist, 0),
             arrowprops=dict(arrowstyle= '|-|', color='tab:red', lw=3)
           )
if True:
    ax.spines['bottom'].set_position('zero')
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.get_xaxis().set_ticklabels([])
    ax.set_xlim(-lim,lim)
    ax.set_ylim(-lim,lim)
    pass

# 2D - density
ax = fig.add_subplot(332, projection='3d')
light = LightSource(90, 45)
illuminated_surface = light.shade(zz_2d, cmap=cm.viridis)
ax.plot_surface(xx_2d, yy_2d, zz_2d, rstride=1, cstride=1, linewidth=0, antialiased=False,
                facecolors=illuminated_surface)
ax.set_axis_off()
ax.set_title("2D")
    
# 2D - field
ax = fig.add_subplot(335)
ax.scatter(X_2d[0], X_2d[1], c=phi_2d, s=phi_2d*1000.+0.01, zorder=3)
if True:
    ax.set_xlim(-lim,lim)
    ax.set_ylim(-lim,lim)
    ax.set_axis_off()
    pass

# 2D - simplification
ax = fig.add_subplot(338)
cercle = build_cercle_xy(sig_gain, 100)[:2]
elipse = cov_2d @ cercle
ax.plot(elipse[0], elipse[1], color='tab:red')
if True:
    ax.set_xlim(-lim,lim)
    ax.set_ylim(-lim,lim)
    ax.set_axis_off()
    pass
    
# 3D - density
ax = fig.add_subplot(333)

ax.text(0.5, 0.5, 'sorry, \nno 4D graph!', horizontalalignment='center',
         verticalalignment='center', transform=ax.transAxes)
ax.set_axis_off()
ax.set_title("3D")

# 3D - density
ax = fig.add_subplot(336, projection='3d')
ax.scatter(X_3d[0], X_3d[1], X_3d[2], s=phi_3d*500000.+0.001, c=phi_3d, zorder=3)
if True:
    ax.set_xlim(-lim,lim)
    ax.set_ylim(-lim,lim)
    ax.set_zlim(-lim,lim)
    ax.set_axis_off()
    pass

# 3D - simplification
ax = fig.add_subplot(339, projection='3d')
n = 50
sphere = build_sphere(sig_gain, n)
ellipsoid = cov_3d @ sphere
ax.plot_surface(np.reshape(ellipsoid[0], (n,n)), 
                np.reshape(ellipsoid[1], (n,n)),
                np.reshape(ellipsoid[2], (n,n)),
                color='tab:red',
                linewidth=0.0, cstride=2, rstride=2
               )
if True:
    ax.set_xlim(-lim,lim)
    ax.set_ylim(-lim,lim)
    ax.set_zlim(-lim,lim)
    ax.set_axis_off()
    pass

fig.tight_layout()

Here are few notes on these representations:
- sometimes people focuses too much on the simplified representation and forget that values are possible outside the borders.
For example in 2D, a Gaussian is not a circle!
- when using the simplified representation, there is not convention on where to draw the border.
The most probable threshold are one-$\sigma$ and three-$\sigma$, but really, it can be everything.
So, when you see these representations, it's important to not assume anything and search for more details.
- the dense field representation, sometimes displayed as an image without the scale changing the size of the pixels, help to understand better that normal distribution is continuous.
Unfortunately, it is way more data to process.
- the typical bell curve can only be displayed in 1D and 2D.

# Covariance

We will take a closer look at the covariance matrix $\matsym{\Sigma}$ as we will need it to extract structural information from a point cloud in the next lesson.
The first explanation is more classical by relying on statistical concepts seen in the [last lesson](5.1-lesson_descriptive_statistics.ipynb).
The second explanation, I find more interesting, uses geometry to give a better intuition of that matrix and uses concept seen in the module [How hard can it be in 3D?](../3-transformations_3d/3.0-overview.ipynb).

## Statistic interpretation

Without any surprise, the covariance matrix $\matsym{\Sigma}$ is the extension of the variance $\sigma^2$ we saw in the last lesson.
Now that we have multiple dimensions to deal with, let's explicit our set of coordinates such that

\begin{aligned}
\mathcal{A} = \{\mat{a}_1, \mat{a}_2, \dots, \mat{a}_i\}
\textdot
\end{aligned}

We will abuse the notation a bit an write that $\mathcal{A}_\axi$ is a subset of $\mathcal{A}$ containing only scalars representing $\axi$-coordinates, such that

\begin{aligned}
\mathcal{A}_\axi = \{a_{1\axi}, a_{2\axi}, \dots, a_{i\axi}\}
\textcomma
\end{aligned}

with $\mathcal{A}_\ayi$ and $\mathcal{A}_\azi$ being defined similarly.
With that in mind, we can explicit each element of the covariance matrix as

\begin{aligned}
\matsym{\Sigma} = \bmat{
\var(\mathcal{A}_\axi) & 
\color{LightSalmon}{\var(\mathcal{A}_\axi, \mathcal{A}_\ayi)} & 
\color{PaleGreen}{\var(\mathcal{A}_\axi, \mathcal{A}_\azi) }
\\
\color{LightSalmon}{\var(\mathcal{A}_\axi, \mathcal{A}_\ayi)} & 
\var(\mathcal{A}_\ayi) &
\color{PaleVioletRed}{\var(\mathcal{A}_\ayi, \mathcal{A}_\azi) }
\\
\color{PaleGreen}{\var(\mathcal{A}_\axi, \mathcal{A}_\azi) } &
\color{PaleVioletRed}{\var(\mathcal{A}_\ayi, \mathcal{A}_\azi) } &
\var(\mathcal{A}_\azi) \\
}
\textdot
\end{aligned}

A covariance matrix must be _symmetric_ and _positive semi-definite_.
The symmetric part should be obvious by looking at the colors and the symmetry over the diagonal elements.
The positive semi-definite part is very specific and simply state that the scalar $s$ from the product

\begin{aligned}
s = \mat{a}^T \matsym{\Sigma} \mat{a}
\end{aligned}

cannot be negative.
In other words, the constraint is that $s \geq 0$ for all $\mat{a} \in \real^k$.
If it looks too complicated, just recall that you can sum two covariance matrices, but you cannot multiply them and still expect that the result is another covariance matrix.

Now, let investigate how to compute each element of the covariance matrix.
Recall the equation for the variance in 1D, for example only using $\mathcal{A}_\axi$, such that

\begin{aligned}
\text{Var}(\mathcal{A}_\axi) 
&= \frac{1}{n}\sum_i^n (a_{i\axi} - \mu_\axi)^2 \\
&= \frac{1}{n}\sum_i^n (a_{i\axi} - \mu_\axi) (a_{i\axi} - \mu_\axi)
\textdot
\end{aligned}

The cross-covariance, for example between $\mathcal{A}_\axi$ and $\mathcal{A}_\ayi$, is very similar and defined as

\begin{aligned}
\var(\mathcal{A}_\axi, \mathcal{A}_\ayi) 
&= \frac{1}{n}\sum_i^n (a_{i\axi} - \mu_\axi) (a_{i\ayi} - \mu_\ayi)
\textdot
\end{aligned}

A cross-covariance value close to zero means that $\mathcal{A}_\axi$ and $\mathcal{A}_\ayi$ are independent.
Also, from the last equation, is should be clear why the covariance matrix is symmetric as

\begin{aligned}
\var(\mathcal{A}_\axi, \mathcal{A}_\ayi) 
&= \frac{1}{n}\sum_i^n (a_{i\axi} - \mu_\axi) (a_{i\ayi} - \mu_\ayi) \\
&= \frac{1}{n}\sum_i^n (a_{i\ayi} - \mu_\ayi) (a_{i\axi} - \mu_\axi)  \\
&= \var(\mathcal{A}_\ayi, \mathcal{A}_\axi)  
\textdot
\end{aligned}

Instead of computing $\matsym{\Sigma}$ element per element, we can first compute an intermediate error matrix $\mat{E}$, such that

\begin{aligned}
\mat{E} = \bmat{(\mat{a}_1 - \mu) & (\mat{a}_2 - \mu) & \cdots & (\mat{a}_i - \mu) &}
\textcomma
\end{aligned}

where $\mat{A} \in \real^{k \times n}$.
Recall that $k$ is the dimension and $n$ the number of points in the set.
Then, you can directly do

\begin{aligned}
\matsym{\Sigma} = \frac{1}{n}\mat{E} \mat{E}^T
\textdot
\end{aligned}

## Geometric interpretation

A covariance matrix is also an affine transformation.
More surprisingly, we can easily express that affine transformation as a chain of basic transformations.
Let's see how.
Generally speaking, we can use eigendecomposition to factorize a square matrix as follow

\begin{aligned}
\mat{A} 
&= \mat{Q}  \matsym{\Lambda} \mat{Q}^{-1}
\textcomma
\end{aligned}

where the diagonal elements of $\matsym{\Lambda}$ hold the eigenvalues and each column of $\mat{Q}$ holds the eigenvector.

**Side note**: what is going on with those words? 
Since when can we make long words like eigendecomposition, eigenvalues, and eigenvector by just writing them together?
It turns our that _eigen_ means peculiar in German and its very common in that language to just compound words.
For example, that's a single word: _Rindfleischetikettierungsüberwachungsaufgabenübertragungsgesetz_.

Now back to our covariance matrix $\matsym{\Sigma}$.
For the specific case of a symmetric and positive semi-definite matrix, it turns out that $\mat{Q}$ is a rotation matrix.
So the covariance matrix is actually chained transformation of a rotation, a scaling, and the same rotation in the opposite direction, such that

\begin{aligned}
\matsym{\Sigma} 
&= \R{} \matsym{\Lambda} \R{}^T \\
&= \R{} \bmat{\lambda_1 & 0 & 0 \\ \ 0 & \lambda_2 & 0 \\ \ 0 & 0 & \lambda_3} \R{}^T \\
\end{aligned}

To understand better what those transformations are doing, we can plot them step by step.
For the following sequence of graphs, I used a point cloud $\mat{P}$ forming a unit circle.
The first point of this point cloud is draw larger to better track rotations.

In [ ]:
# A covariance matrix
Sigma = np.array([[ 0.95, -0.35],
                  [-0.35,  0.95]])

# eigen decomposition
lambdas, R = np.linalg.eig(Sigma)

# build a point cloud shape as a circle
P = build_cercle_xy(radius=1, res=100)[:2]

#--------------------------------
# plotting
if 'fig' in globals():
    plt.close(fig)
fig = plt.figure(figsize=(8,2))

ax = fig.add_subplot(141)
ax.plot(P[0],P[1])
ax.scatter(P[0,0],P[1,0])
ax.set_title(r"$\mathbf{P}$")

ax = fig.add_subplot(142)
P = R.T @ P
ax.plot(P[0],P[1])
ax.scatter(P[0,0],P[1,0])
ax.set_title(r"$\mathbf{R}^T \mathbf{P}$")

ax = fig.add_subplot(143)
P = np.diagflat(lambdas) @ P
ax.plot(P[0],P[1])
ax.scatter(P[0,0],P[1,0])
ax.set_title(r"$\mathbf{\Lambda}\mathbf{R}^T \mathbf{P}$")

ax = fig.add_subplot(144)
P = R @ P
ax.plot(P[0],P[1])
ax.scatter(P[0,0],P[1,0])
ax.set_title(r"$\mathbf{R}\mathbf{\Lambda}\mathbf{R}^T \mathbf{P}$")

lim = 2
for ax in fig.get_axes():
    ax.set_xlim(-lim,lim)
    ax.set_ylim(-lim,lim)
    
fig.tight_layout()

Neat isn't it?
Now you know how those ellipses are drawn to represent a 2D Gaussian and you can build a covariance matrix without data.
You just need to build a valid rotation matrix, which you should be able to do in 2D and 3D by now, and a scaling matrix.
This is what was coded in the large graph with multiple representations of normal distributions.
You can go back to the code and see if you can understand better how the covariance matrices were constructed.

# Conclusion

Actions:
- run the code in your notebook and play with the parameters to understand the behavior of the computation show as examples;
- modify the markdown by adding your own notes using `> my notes` to help you review the material later; and
- complete the tables [Symbol definitions](#Symbol-definitions) and [Glossary](#Glossary) and add your own definitions.

Next lesson:
- [Differential geometry](5.3-lesson_diff_geo.ipynb)

## Symbol definitions

| Symbol             | Definition            |
|--------------------|-------------          |
| $\mathcal{X}, \mathcal{A}$      | sets                  |
| $|\mathcal{X}|$    | cardinality of a set (i.e., how many elements in a set) |
| $x$                | scalar                |
| $\mat{x}$          | coordinates of a vector                |
| $x_i$              | $i^{\text{th}}$ scalar of a set                |
| $\real$            | real set              |
| $\normal$       | normal distribution              |
| $\var(\mathcal{A})$       | variance of $\mathcal{A}$             |
| $\det(\mat{A})$       | determinant of $\mat{A}$             |
| $\matsym{\mu}$       | multivariate mean            |
| $\matsym{\Sigma}$       | covariance matrix            |
| $\in$              | ... is part of ...           |
| ...             |            |

## Glossary

| English             | Français             | Definition |
|-----------          |------------          |:--------   |
| eigenvalues         | valeurs propres      |            |
| eigenvectors        | vecteurs propres     |            |
| Gaussian function   | fonction gaussienne  |            |
| ...                 |                      |            |